# Getting Financial Data - Pandas Datareader

### Introduction:

This time you will get data from a website.


### Step 1. Import the necessary libraries

In [1]:
import pandas as pd
import datetime
import pandas_datareader.data as web
import requests_cache

### Step 2. Create your time range (start and end variables). The start date should be 01/01/2015 and the end should today (whatever your today is).

In [2]:
past = (datetime.datetime.strptime("01/01/2015", "%m/%d/%Y")).date()
present = datetime.date.today()
pd.date_range(past, present).tolist()[:5]

[Timestamp('2015-01-01 00:00:00', freq='D'),
 Timestamp('2015-01-02 00:00:00', freq='D'),
 Timestamp('2015-01-03 00:00:00', freq='D'),
 Timestamp('2015-01-04 00:00:00', freq='D'),
 Timestamp('2015-01-05 00:00:00', freq='D')]

### Step 3. Get an API key for one of the APIs that are supported by Pandas Datareader, preferably for AlphaVantage.

If you do not have an API key for any of the supported APIs, it is easiest to get one for [AlphaVantage](https://www.alphavantage.co/support/#api-key). (Note that the API key is shown directly after the signup. You do *not* receive it via e-mail.)

(For a full list of the APIs that are supported by Pandas Datareader, [see here](https://pydata.github.io/pandas-datareader/readers/index.html). As the APIs are provided by third parties, this list may change.)

### Step 4. Use Pandas Datarader to read the daily time series for the Apple stock (ticker symbol AAPL) between 01/01/2015 and today, assign it to df_apple and print it.

In [3]:
API_KEY = "1OK7XG1VRHY888P3" 
df_apple = web.DataReader('AAPL', "av-daily", start=past, end=present, api_key=API_KEY)

In [4]:
df_apple.head()

,open,high,low,close,volume
2015-01-02,111.39,111.44,107.350,109.33,53204626
2015-01-05,108.29,108.65,105.410,106.25,64285491
2015-01-06,106.54,107.43,104.630,106.26,65797116
2015-01-07,107.20,108.20,106.695,107.75,40105934
2015-01-08,109.23,112.15,108.700,111.89,59364547


### Step 5. Add a new column "stock" to the dataframe and add the ticker symbol

In [5]:
df_apple["stock"] = "AAPL"

### Step 6. Repeat the two previous steps for a few other stocks, always creating a new dataframe: Tesla, IBM and Microsoft. (Ticker symbols TSLA, IBM and MSFT.)

In [6]:
# Caching
expire_after = datetime.timedelta(days=3)
session = requests_cache.CachedSession(cache_name="cache", backend="sqlite", expire_after=expire_after)

In [7]:
ticker_symbols = ["TSLA", "IBM", "MSFT"]
df_name = ["df_tesla", "df_ibm", "df_microsoft"]

for name, ticker in zip(df_name, ticker_symbols):
    vars()[name] = web.DataReader(ticker, "av-daily", start=past, end=present, api_key=API_KEY, session=session)
    vars()[name]["stock"] = ticker

### Step 7. Combine the four separate dataFrames into one combined dataFrame df that holds the information for all four stocks

In [8]:
df = [df_apple, df_tesla, df_ibm, df_microsoft]
combined = pd.concat(df)
combined.rename_axis("Date", inplace=True) # Rename index to "Date"
combined.index = pd.to_datetime(combined.index) # Convert index to datetime

### Step 8. Shift the stock column into the index (making it a multi-level index consisting of the ticker symbol and the date).

In [9]:
combined.set_index("stock", append=True, inplace=True)
combined

,,open,high,low,close,volume
Date,stock,,,,,
2015-01-02,AAPL,111.39,111.44,107.3500,109.33,53204626
2015-01-05,AAPL,108.29,108.65,105.4100,106.25,64285491
2015-01-06,AAPL,106.54,107.43,104.6300,106.26,65797116
2015-01-07,AAPL,107.20,108.20,106.6950,107.75,40105934
2015-01-08,AAPL,109.23,112.15,108.7000,111.89,59364547
...,...,...,...,...,...,...
2021-01-15,MSFT,213.52,214.51,212.0300,212.65,31746512
2021-01-19,MSFT,213.75,216.98,212.6300,216.44,30480859
2021-01-20,MSFT,217.70,225.79,217.2908,224.34,37777260


### Step 9. Create a dataFrame called vol, with the volume values.

In [10]:
vol = combined["volume"]
vol.head()

Date        stock
2015-01-02  AAPL     53204626
2015-01-05  AAPL     64285491
2015-01-06  AAPL     65797116
2015-01-07  AAPL     40105934
2015-01-08  AAPL     59364547
Name: volume, dtype: int64

### Step 10. Aggregate the data of volume to weekly.
Hint: Be careful to not sum data from the same week of 2015 and other years.

In [11]:
bi_weekly = vol.groupby([pd.Grouper(level='stock'), 
            pd.Grouper(level='Date', freq='W')]
          ).sum()
pd.DataFrame(bi_weekly)

volume
stock Date                 
AAPL  2015-01-04   53204626
      2015-01-11  282868187
      2015-01-18  304226647
      2015-01-25  198737041
      2015-02-01  465842684
...                     ...
TSLA  2020-12-27  165945448
      2021-01-03  146960321
      2021-01-10  252137795
      2021-01-17  208896134
      2021-01-24   91697493

[1268 rows x 1 columns]

### Step 11. Find all the volume traded in the year of 2015

In [12]:
reset_weekly = bi_weekly.reset_index(drop=False)
reset_weekly[(reset_weekly["Date"] >= "2015/01/01") & (reset_weekly["Date"] < "2016/01/01")]

,stock,Date,volume
0,AAPL,2015-01-04,53204626
1,AAPL,2015-01-11,282868187
2,AAPL,2015-01-18,304226647
3,AAPL,2015-01-25,198737041
4,AAPL,2015-02-01,465842684
...,...,...,...
998,TSLA,2015-11-29,10946624
999,TSLA,2015-12-06,14888403
1000,TSLA,2015-12-13,14230030
1001,TSLA,2015-12-20,16493091
